In [2]:
import os
import toml
import scipy.stats

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# from statsmodels.stats.multicomp import pairwise_tukeyhsd

from src import settings
from src.utils import fileio

CONFIG_PATH = os.path.join(settings.CONFIG_DIR, "main.toml")
with open(CONFIG_PATH, "r") as file:
    config = toml.load(file)
    
INPUT_PATH = os.path.join(settings.RESULTS_DIR, "distances_traveled")

all_treatments = fileio.load_multiple_folders(INPUT_PATH)
all_treatments = {key: value for key, value in all_treatments.items() if key in config["TREATMENTS"]}

treatment_dataframes = []
for treatment_name, treatment_path in all_treatments.items():
    all_groups = fileio.load_files_from_folder(treatment_path)
    group_dataframes = []
    for group_name, df_path in all_groups.items():
        df = pd.read_csv(df_path, index_col=0)  
        df['Snapshot'] = (df.index / config["FPS"] / config["TIME_WINDOW"]).astype(int) + 1
        df['Group'] = group_name.replace(".csv","")
        group_dataframes.append(df)

    df = pd.concat(group_dataframes)
    df.set_index('Snapshot', inplace=True)
    df.set_index('Group', append=True, inplace=True)
    df['Treatment'] = treatment_name
    df.set_index('Treatment', append=True, inplace=True)
    df = df.reorder_levels(['Treatment', 'Group', 'Snapshot'])
  
    treatment_dataframes.append(df)


combined_data = pd.concat(treatment_dataframes)
combined_data_reset = combined_data.groupby(['Treatment', 'Group']).sum()
combined_data_reset = combined_data_reset.stack().reset_index().rename(columns={'level_2': 'Fly', 0: 'Distance'})
combined_data_reset['Distance'] = combined_data_reset['Distance'] / config['EXPERIMENT_DURATION']

treatment_sums = {}
# for treatment in config["TREATMENTS"]:
   
sns.violinplot(data=combined_data_reset, x='Treatment', y='Distance', hue='Treatment', dodge=False)
# sns.pointplot(data=combined_data_reset, x='Treatment', y='Distance', color='black', markers='o', linestyles='', estimator='mean')
# sns.stripplot(data=combined_data_reset, x='Treatment', y='Distance', hue='Treatment', alpha=0.5)

plt.title('Movement speed per treatment')
# plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), title="Treatment")
plt.xlabel("Treatment name")
plt.ylabel("Movement (mm/s)")
plt.xticks(rotation=45)
plt.legend().remove()
plt.ylim(0, 15)
plt.show()

SystemExit: /srv/milky/drosophila-datasets/fragileX/data/results/distances_traveled is invalid path!

/home/milky/Envs/fragileX/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [4]:
all_groups

NameError: name 'all_groups' is not defined